In [3]:
# default_exp algo.rs.search.vector.faiss

%reload_ext autoreload
%autoreload 2

In [ ]:
algo_rs_search_vector_faiss

# Faiss
https://github.com/facebookresearch/faiss

Faiss is a library for efficient similarity search and clustering of dense vectors. 

Faiss是一个高效相似度搜索和稠密向量聚类的库。

It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Faiss is written in C++ with complete wrappers for Python/numpy. 

它包含可搜索任意大小的向量集的算法，最多可搜索不适合RAM的向量。 它还包含用于评估和参数调整的支持代码。 Faiss用C ++编写，带有完整的Python / numpy包装器。

Some of the most useful algorithms are implemented on the GPU. It is developed by Facebook AI Research.

## How Faiss works

Faiss is built around an index type that stores a set of vectors, and provides a function to search in them with L2 and/or dot product vector comparison. Some index types are simple baselines, such as exact search. Most of the available indexing structures correspond to various trade-offs with respect to

    search time
    search quality
    memory used per index vector
    training time
    need for external data for unsupervised training

The optional GPU implementation provides what is likely (as of March 2017) the fastest exact and approximate (compressed-domain) nearest neighbor search implementation for high-dimensional vectors, fastest Lloyd's k-means, and fastest small k-selection algorithm known. 

# Getting-started
https://github.com/facebookresearch/faiss/wiki/Getting-started

Faiss处理固定维数d（通常为10到100 s）的向量的集合。 这些集合可以存储在矩阵中。 我们假设行主要存储，即向量编号i的第j个分量存储在矩阵的第i行第j列。 Faiss仅使用32位浮点矩阵。

Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s. These collections can be stored in matrices. We assume row-major storage, ie. the j'th component of vector number i is stored in row i, column j of the matrix. Faiss uses only 32-bit floating point matrices.

## Getting some data
We need two matrices:

    xb for the database, that contains all the vectors that must be indexed, and that we are going to search in. Its size is nb-by-d
    xq for the query vectors, for which we need to find the nearest neighbors. Its size is nq-by-d. If we have a single query vector, nq=1.

在以下示例中，我们将使用以d = 64维的均匀分布形式绘制的矢量。 只是为了好玩，我们在第一维上添加了取决于向量索引的小平移。

In the following examples we are going to work with vectors that are drawn form a uniform distribution in d=64 dimensions. Just for fun, we add small translation along the first dimension that depends on the vector index.

In Python, the matrices are always represented as numpy arrays. __The data type dtype must be float32__.

In [7]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
# xb = np.random.random((nb, d))
# xq = np.random.random((nq, d))
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq[:, 0] += np.arange(nq) / 1000.

## Building an index and adding the vectors to it
Faiss围绕Index对象构建。 它封装了数据库向量集，并可选地对其进行预处理以提高搜索效率。 索引的类型很多，我们将使用对它们执行暴力L2距离搜索的最简单版本：IndexFlatL2。

在我们的例子中，所有索引都需要知道何时建立索引，即它们操作的向量的维数。 然后，大多数索引还需要训练阶段，以分析向量的分布。 对于IndexFlatL2，我们可以跳过此操作。

构建和训练索引后，可以对索引执行两项操作：add和search。

要将元素添加到索引，我们在xb上调用add。 我们还可以显示索引的两个状态变量：is_trained，一个布尔值，指示是否需要训练，以及ntotal，即索引向量的数量。

一些索引还可以存储与每个向量相对应的整数ID（但不能存储IndexFlatL2）。 如果未提供ID，则add仅将向量序数用作ID，即。 第一个向量为0，第二个为1，依此类推。

In [8]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


## Searching
可以对索引执行的基本搜索操作是k最近邻搜索，即k。 对于每个查询向量，在数据库中找到它的k个最近邻居。

此操作的结果可以方便地存储在大小为nq-k的整数矩阵中，其中第i行包含查询向量i邻居的ID（按距离递增排序）。 除了此矩阵之外，搜索操作还返回具有相应平方距离的nq-k浮点矩阵。

作为健全性检查，我们可以首先搜索一些数据库向量，以确保最近的邻居确实是向量本身。


In [7]:
np.sum(np.square(xb[393] - xb[0]))

7.1751738

In [3]:
k = 4                          # we want to see 4 nearest neighbors
# 找出和xb中前5个向量 最相近的4个
D, I = index.search(xb[:5], k) # sanity check
print(I)
"""
[[  0 393 363  78]  # xb中和xb[0]最相近的4个向量的索引，第一个当然是它自己
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
"""
print(D)
# np.sum(np.square(xb[393] - xb[0])) # 7.1751738
"""
[[0.        7.1751733 7.207629  7.2511625] # 每个向量和xb[0]的L2距离(平方和)，
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]
"""
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


# Faiss indexes
https://github.com/facebookresearch/faiss/wiki/Faiss-indexes
## IndexFlatL2
Exact Search for L2 
## IndexFlatIP
Exact Search for Inner Product

In [8]:
import faiss                   # make faiss available
index = faiss.IndexFlatIP(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [9]:
from datetime import datetime

In [11]:
xb[0].dot(xb[99415])

40.843704

In [10]:
k = 4                          # we want to see 4 nearest neighbors
# 找出和xb中前5个向量 最相近的4个
D, I = index.search(xb[:5], k) # sanity check
print(I)
"""
[[99415 98718 94699 96532] # xb[0].dot(xb[99415]) # 40.843704
 [98709 99761 99849 98847]
 [98847 99761 98069 86253]
 [97095 92061 99415 98994]
 [98847 99849 97601 97347]]
"""
print(D)
# np.sum(np.square(xb[393] - xb[0])) # 7.1751738
"""
[[40.843704 40.580116 40.504456 40.155773]
 [59.272552 59.212288 58.87538  58.83562 ]
 [29.629923 29.424158 29.413244 29.312313]
 [36.619133 36.581078 36.45857  36.2538  ]
 [40.054226 39.87035  39.75753  39.59665 ]]
"""
s = datetime.now()
D, I = index.search(xq, k)     # actual search
s = print(datetime.now()-s)
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[99415 98718 94699 96532]
 [98709 99761 99849 98847]
 [98847 99761 98069 86253]
 [97095 92061 99415 98994]
 [98847 99849 97601 97347]]
[[40.843704 40.580116 40.504456 40.155773]
 [59.272552 59.212288 58.87538  58.83562 ]
 [29.629923 29.424158 29.413244 29.312313]
 [36.619133 36.581078 36.45857  36.2538  ]
 [40.054226 39.87035  39.75753  39.59665 ]]
0:00:04.177036
[[99849 99597 99614 99196]
 [99761 99195 99709 98711]
 [99702 99962 99886 98925]
 [99712 96098 96532 88354]
 [99886 98847 99530 99614]]
[[99920 99965 99969 99826]
 [99826 99920 99986 99978]
 [99920 99826 99965 99874]
 [99932 99920 99978 99905]
 [99826 99986 99932 99978]]


### index vectors for cosine similarity?

    build an index with METRIC_INNER_PRODUCT
    normalize the vectors prior to adding them to the index (with faiss.normalize_L2 in Python)
    normalize the vectors prior to searching them


In [12]:
faiss.normalize_L2(xb)

In [14]:
import pandas as pd

In [23]:
pd.DataFrame(xb).T.iloc[:, :5].apply(lambda s: np.sum(np.square(s))) # L2为1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [24]:
faiss.normalize_L2(xq)

In [26]:
import faiss                   # make faiss available
index = faiss.IndexFlatIP(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [30]:
type(I)

numpy.ndarray

In [33]:
for i, v in enumerate(I[1, 1:501]):
    print(i, v)

0 911
1 142
2 526


In [27]:
k = 4                          # we want to see 4 nearest neighbors
# 找出和xb中前5个向量 最相近的4个
D, I = index.search(xb[:5], k) # sanity check
print(I)
"""
[[   0 1116  363  973]  # xb[0].dot(xb[1116])  0.8443401
 [   1 1063  555  277]
 [   2  304  101   13]
 [   3  768  316  609]
 [   4  600  288  531]]
"""
print(D)
"""
[[1.         0.84434    0.84233594 0.8421705 ]
 [1.0000002  0.85982877 0.8586012  0.84636354]
 [0.9999999  0.8851342  0.86010313 0.8469719 ]
 [0.99999976 0.82810056 0.8179553  0.8166504 ]
 [1.0000001  0.8542251  0.8457198  0.84149927]]
"""
s = datetime.now()
D, I = index.search(xq, k)     # actual search
s = print(datetime.now()-s)
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[   0 1116  363  973]
 [   1 1063  555  277]
 [   2  304  101   13]
 [   3  768  316  609]
 [   4  600  288  531]]
[[1.         0.84434    0.84233594 0.8421705 ]
 [1.0000002  0.85982877 0.8586012  0.84636354]
 [0.9999999  0.8851342  0.86010313 0.8469719 ]
 [0.99999976 0.82810056 0.8179553  0.8166504 ]
 [1.0000001  0.8542251  0.8457198  0.84149927]]
0:00:03.359087
[[ 207  381 1394 1019]
 [ 300  911  142  526]
 [ 838 1541  527  148]
 [ 196  359  184  466]
 [ 526  120  917  765]]
[[12828 12631 12384 11243]
 [11055 12221 10895 12470]
 [14666 11353 11103 13305]
 [14083 12391 15504 12579]
 [11677 13662 10889 13403]]


In [12]:
#export
def say_hello(to):
    """
    
    """
    return f'Hello {to}!'

# nb_export

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted engineering_nbdev.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
